## Analysis

In [2]:
pip install python-dateutil


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [329]:
import json
import pandas as pd
from dateutil.parser import parse
from datasets import load_dataset, Dataset

small_ds = load_dataset("bigcode-data/the-stack-gh-pull-requests", use_auth_token=True, split="train", streaming=True)

In [330]:
import json
import pandas as pd
from dateutil.parser import parse
from datasets import load_dataset, Dataset

small_ds = load_dataset("bigcode-data/the-stack-gh-pull-requests", use_auth_token=True, split="train", streaming=True)

size = 500_000

ds = small_ds.shuffle(seed=0, buffer_size=1_000_000)

# 10k subset of random samples from ds
fianl_ds = list(ds.take(size))
ds = Dataset.from_pandas(pd.DataFrame(data=fianl_ds))

In [332]:
ds = Dataset.from_pandas(pd.DataFrame(data=fianl_ds))

In [365]:
ds

Dataset({
    features: ['pull_request.guid', 'pull_request.code_review_events', 'pull_request.events', 'pull_request.issue_events', 'bucket', '__index_level_0__'],
    num_rows: 500000
})

In [ ]:
import json
import pandas as pd
from dateutil.parser import parse
from datasets import load_dataset, Dataset

small_ds = load_dataset("bigcode-data/the-stack-gh-pull-requests", use_auth_token=True, split="train", streaming=True)

size = 500_000

ds = small_ds.shuffle(seed=0, buffer_size=1_000_000)

# 10k subset of random samples from ds
fianl_ds = list(ds.take(size))
ds = Dataset.from_pandas(pd.DataFrame(data=fianl_ds))

In [335]:
from pprint import pprint

pprint(ds[0])

{'__index_level_0__': 46164,
 'bucket': None,
 'pull_request.code_review_events': None,
 'pull_request.events': '[{"type": "PullRequestEvent", "action": "opened", '
                        '"actor.login": "pull[bot]", "actor.id": 39814207, '
                        '"user.login": null, "user.id": null, "user.type": '
                        'null, "repo.name": "kofj/website", "repo.id": '
                        '158894695, "public": true, "created_at": '
                        '"2020-11-23T05:58:40Z", "org.id": null, "org.login": '
                        'null, "pull_request.id": 525472638, '
                        '"pull_request.number": 2460, "pull_request.state": '
                        '"open", "pull_request.title": "[pull] master from '
                        'kubernetes:master", "pull_request.body": "See Commits '
                        'and Changes for more details.\\n\\n-----\\nCreated by '
                        '[<img src=\\"https://prod.download/pull-18h-svg\\" '
  

In [412]:
# merge all three instances

pull_request_info_cols = [
    "repo.name",
    "repo.id",
    "org.id",
    "public",
    "pull_request.id",
    "pull_request.guid",
    "pull_request.number",
    "pull_request.title",
    "pull_request.body",
    "pull_request.state",
    "pull_request.user.login",
    "pull_request.user.id",
    # add user type
    "pull_request.head.user.type",
    "pull_request.base.user.type",
    "pull_request.created_at",
    "pull_request.closed_at",
    "pull_request.merged_at",
    "pull_request.merged_by.login",
    "pull_request.milestone.title",
    "pull_request.milestone.description",
    "pull_request.milestone.number",
    # commits
    'pull_request.commits',
    'pull_request.additions',
    'pull_request.deletions',
    # changed files
    'pull_request.changed_files',
    "pull_request.comments",
    "pull_request.review_comments",
]

head_info_cols = [
    "pull_request.head.label",
    "pull_request.head.ref",
    "pull_request.head.user.login",
    "pull_request.head.user.type",
    "pull_request.head.repo.owner.login",
    "pull_request.head.repo.owner.type",
    "pull_request.head.repo.license.name",
    "pull_request.head.sha",
    'pull_request.head.repo.name',
    'pull_request.head.repo.owner.login',
    'pull_request.head.repo.homepage',
    'pull_request.head.repo.description',
    'pull_request.head.repo.language',
    'pull_request.head.repo.stargazers_count',
    'pull_request.head.repo.license.name',
    'pull_request.head.repo.default_branch',
    'pull_request.head.repo.private'
]
base_info_cols = [
    "pull_request.base.label",
    "pull_request.base.ref",
    "pull_request.base.sha",
    "pull_request.base.user.login",
    "pull_request.base.user.type",
    "pull_request.base.repo.owner.login",
    "pull_request.base.repo.owner.type",
    "pull_request.base.repo.license.name",
    "pull_request.base.repo.default_branch",
    "pull_request.base.repo.description",
    "pull_request.base.repo.language",
    "pull_request.base.repo.watchers_count",
    "pull_request.base.repo.open_issues_count",
    "pull_request.base.repo.forks_count",
    'pull_request.base.repo.name',
    'pull_request.base.repo.owner.login',
    'pull_request.base.repo.homepage',
    'pull_request.base.repo.description',
    'pull_request.base.repo.language',
    'pull_request.base.repo.stargazers_count',
    'pull_request.base.repo.private',
    'pull_request.comments',
    'pull_request.review_comments',
    'pull_request.label.name',
]

reviews_info = [# review events only
    'actor.login',
    'actor.id',
    'user.login',
    'user.type',
    'review.state',
    'review.id', 
    'review.body', 
    'review.commit_id', 
    'review.submitted_at', 
    'review.author_association',
    "pull_request.state",
    "pull_request.merged",
    "pull_request.merged_by.login",
    "pull_request.merged_by.type",
    # comments
    'comment.id',
    'comment.diff_hunk',
    'comment.body',
    'comment.path',
    'comment.position',
    'comment.original_position',
    'comment.commit_id',
    'comment.original_commit_id',
    'comment.created_at',
    'comment.updated_at',
    'comment.author_association',
    'comment.start_line',
    'comment.original_start_line',
    'comment.start_side',
    'comment.line',
    'comment.original_line',
    'comment.side',
    'comment.in_reply_to_id',]


issues_info = [
 'author',
 'comment',
 'comment_id']
 
event_info = reviews_info + issues_info

def get_event_info(review):
    res = {k: review[k] if k in review else None for k in event_info}
    # for keys in issues_info add prefix issue.
    for k in issues_info:
        res["issue." + k] = res[k]
        del res[k]
    return res

def load_json(data):
    try:
        data = json.loads(data)
        if isinstance(data, dict):
            data = [data]
        return data
    except TypeError:
        return []

def update_datetime(e):
    e["created_at"] = parse(e["created_at"])
    return e

def merge_events(row):
    events = load_json(row["pull_request.events"])
    reviews = load_json(row["pull_request.code_review_events"])
    issues = load_json(row["pull_request.issue_events"])

    assert len(issues) <= 1
    if issues:
        issues_events = issues[0]["events"]
        # for each events in each category group all events sorted by "created_at" in one list
        for e in issues_events:
            e["created_at"] = parse(e["datetime"])
            del e["datetime"]
    else:
        issues_events = []
    events = [update_datetime(e) for e in events]
    reviews = [update_datetime(e) for e in reviews]
    all_events = sorted(
        events + reviews + issues_events,
        key=lambda x: x["created_at"]
    )
    try:
        base_data = events[0] if events else reviews[0]
    except IndexError:
        if issues:
            base_data = issues_events[0]
            first_event = issues[0]["events"][0]
            base_data['pull_request.title'] = first_event["title"]
            print(f'base data keys: {base_data.keys()}')
            base_data["repo.name"] = base_data["repo"]
            base_data["org.id"] = base_data["org"]
            base_data["repo.name"] = base_data["repo"]
            base_data["pull_request.number"] = int(base_data["pull_request"]["number"])
            base_data["pull_request.user.login"] = base_data["pull_request"]["user_login"]
            print("filling PR data from issue event")
        else:
            raise IndexError("No events for PR")
    
    # Initialize with default values
    pr_info = {k: None for k in pull_request_info_cols}
    head_info = {k: None for k in head_info_cols}
    base_info = {k: None for k in base_info_cols}

    # Fill available data
    pr_info.update({k: base_data[k] if k in base_data else None for k in pull_request_info_cols})
    head_info.update({k: base_data[k] if k in base_data else None  for k in head_info_cols })
    base_info.update({k: base_data[k]  if k in base_data else None for k in base_info_cols})

    # each comment should have "comments" and "review_comments" fields with "extra_review_info" field
    comments = [{"type": e["type"],
                "action": e["action"],
                "created_at": e["created_at"],
                **get_event_info(e)} for e in all_events]
    new_row = {"pull_request_info": pr_info, "head_repo_info": head_info, "base_repo_info": base_info, "events": comments}
    return new_row

In [413]:
new_row = merge_events(row)

KeyError: 'repo'

In [411]:
small_ds_2 = ds.select(range(1000))
dd = small_ds_2.map(merge_events)

KeyError: 'repo'

In [405]:
issues_events[0].keys()

dict_keys(['action', 'author', 'comment', 'comment_id', 'description', 'title', 'type', 'created_at'])

In [366]:
row

{'pull_request.guid': 'karen-kua/COVID-19_Tracker/pull/15',
 'pull_request.code_review_events': None,
 'pull_request.events': None,
 'pull_request.issue_events': '{"repo": "karen-kua/COVID-19_Tracker", "org": null, "issue_id": 1018615993, "issue_number": 15, "pull_request": {"number": 15.0, "repo": "COVID-19_Tracker", "user_login": "karen-kua"}, "events": [{"action": "opened", "author": "dependabot[bot]", "comment": null, "comment_id": null, "datetime": "2021-10-06T15:46:43Z", "description": "Bumps [url-parse](https://github.com/unshiftio/url-parse) from 1.4.7 to 1.5.3.\\n<details>\\n<summary>Commits</summary>\\n<ul>\\n<li><a href=\\"https://github.com/unshiftio/url-parse/commit/ad444931666a30bad11472d89a216461cf16cae2\\"><code>ad44493</code></a> [dist] 1.5.3</li>\\n<li><a href=\\"https://github.com/unshiftio/url-parse/commit/c7984617e235892cc22e0f47bb5ff1c012e6e39f\\"><code>c798461</code></a> [fix] Fix host parsing for file URLs (<a href=\\"https://github-redirect.dependabot.com/unshi

In [360]:
issues[0].keys()

dict_keys(['repo', 'org', 'issue_id', 'issue_number', 'pull_request', 'events'])

In [361]:
issues[0]["pull_request"]

{'number': 15.0, 'repo': 'COVID-19_Tracker', 'user_login': 'karen-kua'}

In [351]:
small_ds_2 = ds.select(range(500))

In [398]:
merged_ds = small_ds_2.map(merge_events, remove_columns=["pull_request.events", "pull_request.code_review_events", "pull_request.issue_events", '__index_level_0__','pull_request.guid'])

KeyError: 'events'

In [150]:
merged_ds.push_to_hub("loubnabnl/code_reviews_3")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:11<00:00, 11.45s/it]


In [72]:
ds = load_dataset("loubnabnl/clean_prs2", split="train")
ds

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 676.50it/s]


Dataset parquet downloaded and prepared to /Users/loubnabenallal/.cache/huggingface/datasets/loubnabnl___parquet/loubnabnl--clean_prs2-50c7cc07186d2bb2/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Dataset({
    features: ['bucket', 'pull_request_info', 'head_repo_info', 'base_repo_info', 'events'],
    num_rows: 10000
})

In [123]:
from pprint import pprint
pprint(small_ds[50])

{'__index_level_0__': 1028,
 'bucket': None,
 'pull_request.code_review_events': None,
 'pull_request.events': '[{"type": "PullRequestEvent", "action": "opened", '
                        '"actor.login": "M-Davies", "actor.id": 25231953, '
                        '"user.login": null, "user.id": null, "user.type": '
                        'null, "repo.name": "AdoptOpenJDK/openjdk-build", '
                        '"repo.id": 85294562, "public": true, "created_at": '
                        '"2020-05-28T09:45:30Z", "org.id": 1673867, '
                        '"org.login": "AdoptOpenJDK", "pull_request.id": '
                        '424372800, "pull_request.number": 1787, '
                        '"pull_request.state": "open", "pull_request.title": '
                        '"Revert \'Fire installer failure on all failed '
                        'results\'", "pull_request.body": "* Seems to cause a '
                        'lot of false positives or just doesnt work overall. '
     

In [151]:
pprint(merged_ds[50]["events"])

[{'action': 'opened',
  'actor.id': 25231953,
  'actor.login': 'M-Davies',
  'comment.author_association': None,
  'comment.body': None,
  'comment.commit_id': None,
  'comment.created_at': None,
  'comment.diff_hunk': None,
  'comment.id': None,
  'comment.in_reply_to_id': None,
  'comment.line': None,
  'comment.original_commit_id': None,
  'comment.original_line': None,
  'comment.original_position': None,
  'comment.original_start_line': None,
  'comment.path': None,
  'comment.position': None,
  'comment.side': None,
  'comment.start_line': None,
  'comment.start_side': None,
  'comment.updated_at': None,
  'created_at': datetime.datetime(2020, 5, 28, 9, 45, 30, tzinfo=<UTC>),
  'issue.author': None,
  'issue.comment': None,
  'issue.comment_id': None,
  'pull_request.merged': False,
  'pull_request.merged_by.login': None,
  'pull_request.merged_by.type': None,
  'pull_request.state': 'open',
  'review.author_association': None,
  'review.body': None,
  'review.commit_id': None,
 

In [222]:
ds = load_dataset("loubnabnl/code_reviews_3", split="train")
size = len(ds)

Found cached dataset parquet (/Users/loubnabenallal/.cache/huggingface/datasets/loubnabnl___parquet/loubnabnl--code_reviews_3-c3e4ac735edf14b4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [223]:
sample = ds[1470]
events = sample["events"]
grouped_events = create_grouped_events(events)
original_poster = sample["pull_request_info"]['pull_request.user.login']

In [ ]:
from pprint import pprint

pprint(small_ds[50])

In [224]:
def get_pr_info(sample):
    pr_info = sample["pull_request_info"]
    head_info = sample["head_repo_info"]
    base_info = sample["base_repo_info"]
    events = sample["events"]

    gh_link = f"https://github.com/{pr_info['repo.name']}/pull/{pr_info['pull_request.number']}"

    header = f"""📝 **Title**: {pr_info['pull_request.title']}<br>
    📦 **GitHub Repo**: {pr_info['repo.name']}, PR Number: {pr_info['pull_request.number']}, ID: {pr_info['pull_request.id']}.<br>
    Link: [{gh_link}]({gh_link})"""
    pr_info_html = f"""
    <table style="width:100%">
        <tr><th>Attribute</th><th>Detail</th></tr>
        <tr><td>🧾 <strong>PR Type</strong></td><td>{events[0]['type']}</td></tr>
        <tr><td>🟢 <strong>PR State</strong></td><td>{pr_info['pull_request.state']}</td></tr>
        <tr><td>👤 <strong>PR Author</strong></td><td>{pr_info['pull_request.user.login']}</td></tr>
        <tr><td>🏷️ <strong>Head Branch</strong></td><td>ref: {head_info['pull_request.head.ref']}, label: {head_info['pull_request.head.label']}</td></tr>
        <tr><td>🌳 <strong>Base Branch</strong></td><td>{base_info['pull_request.base.ref']}</td></tr>
    </table>
    """
    return header, pr_info_html, pr_info['pull_request.body']

from IPython.display import HTML, display
display(HTML(get_pr_info(sample)[0]))
display(HTML(get_pr_info(sample)[1]))
display(HTML(get_pr_info(sample)[2]))

Attribute,Detail
🧾 PR Type,issue
🟢 PR State,open
👤 PR Author,kasobol-msft
🏷️ Head Branch,"ref: kasobol-msft-patch-1, label: Azure:kasobol-msft-patch-1"
🌳 Base Branch,main


In [308]:
sample = ds[4]
events = sample["events"]

In [309]:
len(events)

2

In [310]:
pprint(events)

[{'action': 'opened',
  'actor.id': 39814207,
  'actor.login': 'pull[bot]',
  'comment.author_association': None,
  'comment.body': None,
  'comment.commit_id': None,
  'comment.created_at': None,
  'comment.diff_hunk': None,
  'comment.id': None,
  'comment.in_reply_to_id': None,
  'comment.line': None,
  'comment.original_commit_id': None,
  'comment.original_line': None,
  'comment.original_position': None,
  'comment.original_start_line': None,
  'comment.path': None,
  'comment.position': None,
  'comment.side': None,
  'comment.start_line': None,
  'comment.start_side': None,
  'comment.updated_at': None,
  'created_at': datetime.datetime(2022, 10, 10, 10, 57, 41, tzinfo=<UTC>),
  'issue.author': None,
  'issue.comment': None,
  'issue.comment_id': None,
  'pull_request.merged': False,
  'pull_request.merged_by.login': None,
  'pull_request.merged_by.type': None,
  'pull_request.state': 'open',
  'review.author_association': None,
  'review.body': None,
  'review.commit_id': None

In [ ]:

import uuid

def create_grouped_events(events):
    df = pd.DataFrame(events)
    # Ensure it's in datetime format
    df['created_at'] = pd.to_datetime(df['created_at'])
    # Create a new column 'uuid' initialized with None
    df['uuid'] = None
    # For rows where either 'comment.diff_hunk' or 'comment.commit_id' is NaN, assign a unique UUID
    mask = df['comment.diff_hunk'].isna() | df['comment.commit_id'].isna()
    df.loc[mask, 'uuid'] = [str(uuid.uuid4()) for _ in range(mask.sum())]
    # Group by 'comment.diff_hunk', 'comment.commit_id', and 'uuid'
    grouped_events = [group.drop(columns='uuid').to_dict(orient='records') for _, group in df.groupby(['comment.diff_hunk', 'comment.commit_id', 'uuid'], dropna=False)]
    return grouped_events



In [229]:
for i in range(len(ds)):
    e = ds[i]
    if e["events"][0]["comment.diff_hunk"]:
        print(i)
        break

846


In [299]:
pprint(events)

[{'action': 'opened',
  'actor.id': 1753262,
  'actor.login': 'mo9a7i',
  'comment.author_association': None,
  'comment.body': None,
  'comment.commit_id': None,
  'comment.created_at': None,
  'comment.diff_hunk': None,
  'comment.id': None,
  'comment.in_reply_to_id': None,
  'comment.line': None,
  'comment.original_commit_id': None,
  'comment.original_line': None,
  'comment.original_position': None,
  'comment.original_start_line': None,
  'comment.path': None,
  'comment.position': None,
  'comment.side': None,
  'comment.start_line': None,
  'comment.start_side': None,
  'comment.updated_at': None,
  'created_at': datetime.datetime(2022, 5, 5, 4, 35, 2, tzinfo=<UTC>),
  'issue.author': None,
  'issue.comment': None,
  'issue.comment_id': None,
  'pull_request.merged': False,
  'pull_request.merged_by.login': None,
  'pull_request.merged_by.type': None,
  'pull_request.state': 'open',
  'review.author_association': None,
  'review.body': None,
  'review.commit_id': None,
  'rev

In [303]:
import numpy as np
df = pd.DataFrame(events)
df['created_at'] = pd.to_datetime(df['created_at'])
df.drop_duplicates(inplace=True)
# Create a new 'group_key' column. For non-null 'review.commit_id' values, it's the same value.
mask = df['review.commit_id'].isnull()
df.loc[mask, 'group_key'] = np.arange(mask.sum()) + 1
df.loc[~mask, 'group_key'] = df['review.commit_id']
df

,action,actor.id,actor.login,comment.author_association,comment.body,comment.commit_id,comment.created_at,comment.diff_hunk,comment.id,comment.in_reply_to_id,...,review.author_association,review.body,review.commit_id,review.id,review.state,review.submitted_at,type,user.login,user.type,group_key
0,opened,1753262,mo9a7i,None,None,None,None,None,None,None,...,None,None,None,NaN,None,None,PullRequestEvent,None,None,1.0
1,created,1753262,mo9a7i,None,None,None,None,None,None,None,...,MEMBER,looks fine,ba75444d1ada77cf5f3f06cd74b6320bab8db54b,962846794.0,commented,2022-05-05T04:35:02Z,PullRequestReviewEvent,mo9a7i,User,ba75444d1ada77cf5f3f06cd74b6320bab8db54b
2,closed,1753262,mo9a7i,None,None,None,None,None,None,None,...,None,None,None,NaN,None,None,PullRequestEvent,None,None,2.0


In [304]:
import numpy as np
df = pd.DataFrame(events)
df['created_at'] = pd.to_datetime(df['created_at'])
df.drop_duplicates(inplace=True)
# Create a new 'group_key' column. For non-null 'review.commit_id' values, it's the same value.
mask = df['review.commit_id'].isnull()
df.loc[mask, 'group_key'] = np.arange(mask.sum()) + 1
df.loc[~mask, 'group_key'] = df['review.commit_id']

if len(df) == 1:
    grouped_events = [[df.iloc[0].to_dict()]]
else:
    grouped_events = [group.to_dict(orient='records') for _, group in df.groupby('group_key', dropna=False)]

# sort by first event date
grouped_events = sorted(grouped_events, key=lambda x: x[0]['created_at'])


In [311]:
def create_grouped_events(events):
    """group events that happened in the same  review thread using review.commit_id"""
    df = pd.DataFrame(events)
    df['created_at'] = pd.to_datetime(df['created_at'])
    df.drop_duplicates(inplace=True)
    # Create a new 'group_key' where rows with NaN 'review.commit_id' get an identical identifier. Otherwise NaN values go in the same group
    mask = df['review.commit_id'].isnull()
    df.loc[mask, 'group_key'] = np.arange(mask.sum()) + 1
    df.loc[~mask, 'group_key'] = df['review.commit_id']
    
    if len(df) == 1:
        grouped_events = [[df.iloc[0].to_dict()]]
    else:
        grouped_events = [group.to_dict(orient='records') for _, group in df.groupby('group_key', dropna=False)]
    
    # sort by first event date
    grouped_events = sorted(grouped_events, key=lambda x: x[0]['created_at'])
    return grouped_events

grouped_events = create_grouped_events(events)

In [312]:
print(f"len events {len(events)} and len grouped_events {len(grouped_events)}")

len events 2 and len grouped_events 2


In [313]:

original_poster = sample["pull_request_info"]['pull_request.user.login']
thread_html = ""
c = 0
for thread in grouped_events:
    print(f"thread number {c}")
    c += 1
    thread_html += '<div class="thread">'
    # Get the first event in the thread as a reference
    first_event = thread[0]
    poster_name = first_event['actor.login'] or first_event['issue.author'] or first_event['user.login']
    # Add shared parts of the events only once
    user_type = f"(type :<strong>{first_event['user.type']}</strong>)" if first_event['user.type'] else ""
    review_state = f"<tr><td><strong>Review State</strong></td><td>{first_event['review.state']}</td></tr>" if first_event['review.state'] else ""
    text = f"""
    <div class="event">
        <table style="width:100%; border: 0;">
            <tr><td><strong>Event Type</strong></td><td>{first_event['type']}</td></tr>
            <tr><td><strong>User</strong></td><td>{poster_name} {user_type}</td></tr>
            <tr><td><strong>Action</strong></td><td>{first_event['action']}</td></tr>
            {review_state}
            <tr><td><strong>PR State</strong></td><td>{first_event["pull_request.state"]}, merged: {first_event['pull_request.merged']}</td></tr>
            <tr><td><strong>Date</strong></td><td>{first_event['created_at']}</td></tr>
        </table>
    </div>
    """
    highlight_action = "background-color: #FFCFCF;" if first_event['action'] == 'closed' else ""
    highlight_pr_state = "background-color: #FFCFCF;" if first_event['pull_request.merged'] else ""

    text = f"""
    <div class="event">
        <table style="width:100%; border: 0;">
            <tr><td><strong>Event Type</strong></td><td>{first_event['type']}</td></tr>
            <tr><td><strong>User</strong></td><td>{poster_name} {user_type}</td></tr>
            <tr><td><strong>Action</strong></td><td style="{highlight_action}">{first_event['action']}</td></tr>
            "<tr><td><strong>Review State</strong></td><td>{first_event['review.state']}</td></tr>"
            <tr><td><strong>PR State</strong></td><td style="{highlight_pr_state}">{first_event["pull_request.state"]}, merged: {first_event['pull_request.merged']}</td></tr>
            <tr><td><strong>Date</strong></td><td>{first_event['created_at']}</td></tr>
        </table>
    </div>
    """


    thread_html += text
    thread_html += ("\n" + "-"*165)
    # Add the bodies of the comments for each event in the thread
    for event in thread:
        # from 'actor.login' and 'issue.author' and 'user.login' take which ever isn't none
        poster_name = event['actor.login'] or event['issue.author'] or event['user.login']
        if event['comment.body'] or event["issue.comment"]:
            is_op = original_poster == poster_name
            thread_html += format_body(event['comment.body'], poster_name, is_op)

    thread_html += '</div>'

display(HTML(thread_html))

thread number 0
thread number 1


Event Type,PullRequestEvent
User,pull[bot]
Action,opened
Review State,None
PR State,"open, merged: False"
Date,2022-10-10 10:57:41+00:00
Event Type,PullRequestEvent
User,pull[bot]
Action,closed
Review State,None
PR State,"closed, merged: True"


In [314]:
def display_events(sample):
    events = sample["events"]
    grouped_events = create_grouped_events(events)
    original_poster = sample["pull_request_info"]['pull_request.user.login']
    for thread in grouped_events:
        thread_html = '<div class="thread">'
        # Get the first event in the thread as a reference
        first_event = thread[0]
        poster_name = first_event['actor.login'] or first_event['issue.author'] or first_event['user.login']
        # Add shared parts of the events only once
        user_type = f"(type :<strong>{first_event['user.type']}</strong>)" if first_event['user.type'] else ""
        highlight_action = "background-color: #FFCFCF;" if first_event['action'] == 'closed' else ""
        highlight_pr_state = "background-color: #FFCFCF;" if first_event['pull_request.merged'] else ""
    
        text = f"""
        <div class="event">
            <table style="width:100%; border: 0;">
                <tr><td><strong>Event Type</strong></td><td>{first_event['type']}</td></tr>
                <tr><td><strong>User</strong></td><td>{poster_name} {user_type}</td></tr>
                <tr><td><strong>Action</strong></td><td style='{highlight_action}'>{first_event['action']}</td></tr>
                <tr><td><strong>Review State</strong></td><td>{first_event['review.state']}</td></tr>
                <tr><td><strong>PR State</strong></td><td style="{highlight_pr_state}">{first_event["pull_request.state"]}, merged: {first_event['pull_request.merged']}</td></tr>
                <tr><td><strong>Date</strong></td><td>{first_event['created_at']}</td></tr>
            </table>
        </div>
        """
        print(f"added first event of teh group")
        thread_html += text
        
        # Add the bodies of the comments for each event in the thread
        for event in thread:
            # from 'actor.login' and 'issue.author' and 'user.login' take which ever isn't none
            poster_name = event['actor.login'] or event['issue.author'] or event['user.login']
            if event['comment.body'] or event["issue.comment"]:
                is_op = original_poster == poster_name
                thread_html += format_body(event['comment.body'], poster_name, is_op)

        thread_html += '</div>'
        display(HTML(thread_html))
        if first_event['comment.path']:
            path_html = f"<code style='font-family: monospace; font-weight: bold;'>Path:</code> {first_event['comment.path']}"
            display(HTML(path_html))
        display(HTML("---"))

In [316]:
grouped_events

[[{'action': 'opened',
   'actor.id': 39814207,
   'actor.login': 'pull[bot]',
   'comment.author_association': None,
   'comment.body': None,
   'comment.commit_id': None,
   'comment.created_at': None,
   'comment.diff_hunk': None,
   'comment.id': None,
   'comment.in_reply_to_id': None,
   'comment.line': None,
   'comment.original_commit_id': None,
   'comment.original_line': None,
   'comment.original_position': None,
   'comment.original_start_line': None,
   'comment.path': None,
   'comment.position': None,
   'comment.side': None,
   'comment.start_line': None,
   'comment.start_side': None,
   'comment.updated_at': None,
   'created_at': Timestamp('2022-10-10 10:57:41+0000', tz='UTC'),
   'issue.author': None,
   'issue.comment': None,
   'issue.comment_id': None,
   'pull_request.merged': False,
   'pull_request.merged_by.login': None,
   'pull_request.merged_by.type': None,
   'pull_request.state': 'open',
   'review.author_association': None,
   'review.body': None,
   'r

In [315]:
display_events(sample)

Event Type,PullRequestEvent
User,pull[bot]
Action,opened
Review State,None
PR State,"open, merged: False"
Date,2022-10-10 10:57:41+00:00


Event Type,PullRequestEvent
User,pull[bot]
Action,opened
Review State,None
PR State,"open, merged: False"
Date,2022-10-10 10:57:41+00:00
Event Type,PullRequestEvent
User,pull[bot]
Action,closed
Review State,None
PR State,"closed, merged: True"


In [261]:
for group in grouped_events:
    if len(group) == 1:
        poster_name = group[0]['actor.login'] or group[0]['issue.author'] or group[0]['user.login']
        print("single")
        print(group[0]["created_at"])
        print(f"with review state: {group[0]['review.state']}")
        print(group[0]["comment.body"])
        # print action type and user
        print(f"User: {poster_name}, action: {group[0]['action']}")
        print(group[0]["type"])
        print("------------")
        continue
    # date 
    else:
        print("multiple")
        poster_name = group[0]['actor.login'] or group[0]['issue.author'] or group[0]['user.login']
        print("single")
        print(group[0]["created_at"])
        print(f"with review state: {group[0]['review.state']}")
        print(group[0]["comment.body"])
        # print action type and user
        print(f"User: {poster_name}, action: {group[0]['action']}")
        print(group[0]["type"])
        print("------------")
        print("------------")
        for e in group[1:]:
            print(group[0]["created_at"])
            print(f"with review state: {group[0]['review.state']} PR state {group[0]['pull_request.merged']}")
            print(e["comment.body"])
            poster_name = e['actor.login'] or e['issue.author'] or e['user.login']
            print(f"User: {poster_name}, action: {e['action']}")
            print(e["type"])
            print("------------")
        print("------------end multiple")

single
2022-05-05 04:35:02+00:00
with review state: commented
None
User: mo9a7i, action: created
PullRequestReviewEvent
------------
multiple
single
2022-05-05 04:35:02+00:00
with review state: None
None
User: mo9a7i, action: opened
PullRequestEvent
------------
------------
2022-05-05 04:35:02+00:00
with review state: None PR state False
None
User: mo9a7i, action: closed
PullRequestEvent
------------
------------end multiple


In [225]:
def create_grouped_events(events):
    df = pd.DataFrame(events)
    df['created_at'] = pd.to_datetime(df['created_at'])
    df = df.sort_values(['comment.diff_hunk', 'comment.commit_id', 'created_at'])
    # Group events in a the same thread using 'comment.diff_hunk' and 'comment.commit_id'
    if len(df) == 1:
        grouped_events = [[df.iloc[0].to_dict()]]
    else:
        grouped_events = [group.to_dict(orient='records') for _, group in df.groupby(['comment.diff_hunk', 'comment.commit_id', 'pull_request.state'], dropna=False)]
    return grouped_events

def format_body(text, user, is_op=False):
    color = "#007bff" if is_op else "black"
    pr_body = f"<div style='background-color: #f0f0f0; padding: 10px;'>👤<strong style='color: {color};'>{user}</strong>: {text}</div>"
    return pr_body

In [220]:
import uuid
import pandas as pd

def create_grouped_events(events):
    df = pd.DataFrame(events)
    
    # Ensure it's in datetime format
    df['created_at'] = pd.to_datetime(df['created_at'])
    # Preserve the original order
    df['order'] = range(len(df))

    # Create a new column 'uuid' initialized with None
    df['uuid'] = None

    # For rows where either 'comment.diff_hunk' or 'comment.commit_id' is NaN, assign a unique UUID
    mask = df['comment.diff_hunk'].isna() | df['comment.commit_id'].isna()
    df.loc[mask, 'uuid'] = [str(uuid.uuid4()) for _ in range(mask.sum())]

    # Group by 'comment.diff_hunk', 'comment.commit_id', and 'uuid'
    grouped_events = [group.drop(columns=['uuid', 'order']).to_dict(orient='records') 
                      for _, group in df.sort_values(by='order').groupby(['comment.diff_hunk', 'comment.commit_id', 'uuid'], dropna=False)]
    # soert on created_at
    grouped_events = [sorted(group, key=lambda x: x['created_at']) for group in grouped_events]
    return grouped_events



grouped_events = create_grouped_events(events)
c = 0
thread_html = ""
for thread in grouped_events:
    # Start a new thread
    #print(thread)
    if thread[0]["action"] == "opened":
        continue
    thread_html += '<div class="thread">'
    # Get the first event in the thread as a reference
    first_event = thread[0]
    poster_name = first_event['actor.login'] or first_event['issue.author'] or first_event['user.login']
    # Add shared parts of the events only once
    text = f"""
    <div class="event">
        <table style="width:100%; border: 0;">
            <tr><td><strong>Event Type</strong></td><td>{first_event['type']}</td></tr>
            <tr><td><strong>User</strong></td><td>{poster_name} (type :<strong>{first_event['user.type']}</strong>)</td></tr>
            <tr><td><strong>Action</strong></td><td>{first_event['action']}</td></tr>
            <tr><td><strong>Review State</strong></td><td>{first_event['review.state']}</td></tr>
            <tr><td><strong>PR State</strong></td><td>{first_event["pull_request.state"]}, merged: {first_event['pull_request.merged']}</td></tr>
            <tr><td><strong>From Head</strong></td><td>{sample["head_repo_info"]['pull_request.head.label']}</td></tr>
        </table>
    </div>
    """
    thread_html += text
    # add horizontal line
    thread_html += '<hr>'
    for event in thread:
        # from 'actor.login' and 'issue.author' and 'user.login' take which ever isn't none
        poster_name = event['actor.login'] or event['issue.author'] or event['user.login']
        if event['comment.body'] or event["issue.comment"]:
            is_op = original_poster == poster_name
            thread_html += format_body(event['comment.body'], poster_name, is_op)

    thread_html += '</div>'

In [218]:
len(grouped_events)

4

In [221]:
pprint(grouped_events)

[[{'action': 'created',
   'actor.id': nan,
   'actor.login': None,
   'comment.author_association': None,
   'comment.body': None,
   'comment.commit_id': None,
   'comment.created_at': None,
   'comment.diff_hunk': None,
   'comment.id': None,
   'comment.in_reply_to_id': None,
   'comment.line': None,
   'comment.original_commit_id': None,
   'comment.original_line': None,
   'comment.original_position': None,
   'comment.original_start_line': None,
   'comment.path': None,
   'comment.position': None,
   'comment.side': None,
   'comment.start_line': None,
   'comment.start_side': None,
   'comment.updated_at': None,
   'created_at': Timestamp('2022-02-01 00:05:19+0000', tz='UTC'),
   'issue.author': 'kasobol-msft',
   'issue.comment': "This won't work well because it includes dependencies in "
                    'output like this:\r\n'
                    '![image](https://user-images.githubusercontent.com/61715331/151893024-ef3e99d9-0d83-44c6-839b-966550320642.png)\r\n'
        

In [193]:
from IPython.display import HTML, display
display(HTML(thread_html))

Event Type,PullRequestEvent
User,kasobol-msft (type :None)
Action,closed
Review State,None
PR State,"closed, merged: False"
From Head,Azure:kasobol-msft-patch-1


In [92]:
merged_ds[0]["base_repo_info"]

{'pull_request.base.label': 'AbdElrahmanMuhammedNasr:master',
 'pull_request.base.ref': 'master',
 'pull_request.base.repo.default_branch': 'master',
 'pull_request.base.repo.description': None,
 'pull_request.base.repo.forks_count': 0,
 'pull_request.base.repo.homepage': None,
 'pull_request.base.repo.language': 'TypeScript',
 'pull_request.base.repo.license.name': None,
 'pull_request.base.repo.name': 'WuzuufMasr',
 'pull_request.base.repo.open_issues_count': 24,
 'pull_request.base.repo.owner.login': 'AbdElrahmanMuhammedNasr',
 'pull_request.base.repo.owner.type': 'User',
 'pull_request.base.repo.private': False,
 'pull_request.base.repo.stargazers_count': 0,
 'pull_request.base.repo.watchers_count': 0,
 'pull_request.base.sha': 'a7d0127c02152dca69c41f83afb1a0a4d0c0e004',
 'pull_request.base.user.login': 'AbdElrahmanMuhammedNasr',
 'pull_request.base.user.type': 'User',
 'pull_request.comments': 0,
 'pull_request.label.name': None,
 'pull_request.review_comments': 0}

In [80]:
ds = merged_ds

In [321]:
pprint(small_ds[8])

{'__index_level_0__': 175,
 'bucket': '940',
 'pull_request.code_review_events': None,
 'pull_request.events': '[{"type": "PullRequestEvent", "action": "opened", '
                        '"actor.login": "pkarman", "actor.id": 1205061, '
                        '"user.login": null, "user.id": null, "user.type": '
                        'null, "repo.name": "18F/C2", "repo.id": 18201810, '
                        '"public": true, "created_at": "2015-11-23T19:16:36Z", '
                        '"org.id": 6233994, "org.login": "18F", '
                        '"pull_request.id": 51566831, "pull_request.number": '
                        '820, "pull_request.state": "open", '
                        '"pull_request.title": "rename elk services to '
                        'workaround blue-green deploy bug", '
                        '"pull_request.body": "there\'s a bug in the '
                        'cf-blue-green deploy that gets a false positive match '
                        'based on

In [327]:
actions = []
c = 0
for events in ds["events"]:
    c += 1
    actions.extend([event["action"] for event in events])
    if c > 10000:
        break


In [328]:
set(actions)

{'closed', 'created', 'opened', 'reopened'}

In [322]:
pprint(ds[6])

{'base_repo_info': {'pull_request.base.label': '1011X:master',
                    'pull_request.base.ref': 'master',
                    'pull_request.base.repo.default_branch': 'master',
                    'pull_request.base.repo.description': 'Representing '
                                                          'rational numbers '
                                                          'using the '
                                                          'floating-bar number '
                                                          'type.',
                    'pull_request.base.repo.forks_count': 2,
                    'pull_request.base.repo.homepage': None,
                    'pull_request.base.repo.language': 'Rust',
                    'pull_request.base.repo.license.name': 'Other',
                    'pull_request.base.repo.name': 'floating_bar',
                    'pull_request.base.repo.open_issues_count': 6,
                    'pull_request.base.repo.owner.log

In [318]:
sample = ds[6]
sample

{'bucket': '940',
 'pull_request_info': {'org.id': None,
  'public': True,
  'pull_request.additions': 23,
  'pull_request.base.user.type': 'User',
  'pull_request.body': '',
  'pull_request.changed_files': 4,
  'pull_request.closed_at': None,
  'pull_request.comments': 0,
  'pull_request.commits': 1,
  'pull_request.created_at': '2021-05-08T20:30:31Z',
  'pull_request.deletions': 19,
  'pull_request.guid': '1011X/floating_bar/pull/7',
  'pull_request.head.user.type': 'User',
  'pull_request.id': 634875503,
  'pull_request.merged_at': None,
  'pull_request.merged_by.login': None,
  'pull_request.milestone.description': None,
  'pull_request.milestone.number': None,
  'pull_request.milestone.title': None,
  'pull_request.number': 7,
  'pull_request.review_comments': 0,
  'pull_request.state': 'open',
  'pull_request.title': 'change benches to use fractional values',
  'pull_request.user.id': 8010244,
  'pull_request.user.login': 'ZoeyR',
  'repo.id': 166723951,
  'repo.name': '1011X/flo

In [81]:
sample = ds[0]
pr_info = sample["pull_request_info"]
head_info = sample["head_repo_info"]
base_info = sample["base_repo_info"]
events = sample["events"]

gh_link = f"https://github.com/{pr_info['repo.name']}/pull/{pr_info['pull_request.number']}"

header = f"""📝 **Title**: {pr_info['pull_request.title']}<br>
📦 **GitHub Repo**: {pr_info['repo.name']}, PR Number: {pr_info['pull_request.number']}, ID: {pr_info['pull_request.id']}.<br>
Link: [{gh_link}]({gh_link})"""
pr_info_html = f"""
<table style="width:100%">
    <tr><th>Attribute</th><th>Detail</th></tr>
    <tr><td>🧾 <strong>PR Type</strong></td><td>{events[0]['type']}</td></tr>
    <tr><td>🟢 <strong>PR State</strong></td><td>{pr_info['pull_request.state']}</td></tr>
    <tr><td>👤 <strong>PR Author</strong></td><td>{pr_info['pull_request.user.login']}</td></tr>
    <tr><td>🏷️ <strong>Head Branch</strong></td><td>ref: {head_info['pull_request.head.ref']}, label: {head_info['pull_request.head.label']}</td></tr>
    <tr><td>🌳 <strong>Base Branch</strong></td><td>{base_info['pull_request.base.ref']}</td></tr>
</table>
"""

In [82]:
# display pr_info_html as HTML
from IPython.display import HTML, display
display(HTML(pr_info_html))

Attribute,Detail
🧾 PR Type,PullRequestEvent
🟢 PR State,open
👤 PR Author,dependabot[bot]
🏷️ Head Branch,"ref: dependabot/npm_and_yarn/qs-6.5.3, label: AbdElrahmanMuhammedNasr:dependabot/npm_and_yarn/qs-6.5.3"
🌳 Base Branch,master


In [75]:
df = pd.DataFrame(events)
df

,action,comments,created_at,type
0,opened,"{'actor.id': 49699333, 'actor.login': 'dependa...",2022-12-10 03:27:08+00:00,PullRequestEvent


In [84]:
def create_grouped_events(events):
    df = pd.DataFrame(events)
    df['created_at'] = pd.to_datetime(df['created_at'])
    df = df.sort_values(['comment.diff_hunk', 'comment.commit_id', 'created_at'])
    # Group events in a the same thread using 'comment.diff_hunk' and 'comment.commit_id'
    if len(df) == 1:
        grouped_events = [[df.iloc[0].to_dict()]]
    else:
        grouped_events = [group.to_dict(orient='records') for _, group in df.groupby(['comment.diff_hunk', 'comment.commit_id'])]
    return grouped_events
    
events = sample["events"]
grouped_events = create_grouped_events(events)
original_poster = sample["pull_request_info"]['pull_request.user.login']
for thread in grouped_events:
    # Start a new thread
    thread_html = '<div class="thread">'
    # Get the first event in the thread as a reference
    first_event = thread[0]
    
    # Add shared parts of the events only once
    text = f"""
    <div class="event">
        <table style="width:100%; border: 0;">
            <tr><td><strong>Event Type</strong></td><td>{first_event['type']}</td></tr>
            <tr><td><strong>User</strong></td><td>{first_event['user.login']} (type :<strong>{first_event['user.type']}</strong>)</td></tr>
            <tr><td><strong>Review State</strong></td><td>{first_event['review.state']}</td></tr>
            <tr><td><strong>From Head</strong></td><td>{head_info['pull_request.head.label']}</td></tr>
        </table>
    </div>
    """
    thread_html += text
    
    # Add the bodies of the comments for each event in the thread
    for event in thread:
        if event['comment.body']:
            is_op = original_poster == event['user.login']
            thread_html += format_body(event['comment.body'], event['user.login'], is_op)
    thread_html += '</div>'
    display(HTML(thread_html))
    if first_event['comment.path']:
        path_html = f"<code style='font-family: monospace; font-weight: bold;'>Path:</code> {first_event['comment.path']}"
        display(HTML(path_html))
    if first_event["comment.diff_hunk"]:
        print(first_event["comment.diff_hunk"])

Event Type,PullRequestEvent
User,None (type :None)
Review State,None
From Head,AbdElrahmanMuhammedNasr:dependabot/npm_and_yarn/qs-6.5.3


In [54]:
sample["pull_request_info"]['pull_request.user.login']

'dependabot[bot]'

In [ ]:
import os
import pandas as pd
import ghdiff
import streamlit as st
import streamlit.components.v1 as components
from datasets import load_dataset


# save dataset as in "bigcode/code_reviews_sample"
ds = load_dataset("loubnabnl/clean_prs2", split="train")
size = len(ds)

def show_diff_hunk(diff_hunk, position, context=5):
    # exclude the first line with the @@ notation
    lines = diff_hunk.split('\n')
    start_line = max(int(position) - context - 1, 0)
    end_line = int(position)
    actual_diff = lines[0] + '\n' + '\n'.join(lines[start_line + 1:end_line + 1])
    focus = ghdiff.colorize(actual_diff)
    full = ghdiff.colorize(diff_hunk)
    # Wrap the diff hunk inside a scrollable div
    scrollable_focus = f'<div style="height:500px;overflow:auto;">{focus}</div>'
    scrollable_full = f'<div style="height:500px;overflow:auto;">{full}</div>'
    if len(lines) <= 12:
        return None, scrollable_full
    return scrollable_focus, scrollable_full


def format_body(text, user, is_op=False):
    color = "#007bff" if is_op else "black"
    pr_body = f"<div style='background-color: #f0f0f0; padding: 10px;'>👤<strong style='color: {color};'>{user}</strong>: {text}</div>"
    return pr_body


def create_grouped_events(events):
    df = pd.DataFrame(events)
    df['created_at'] = pd.to_datetime(df['created_at'])
    df = df.sort_values(['comment.diff_hunk', 'comment.commit_id', 'created_at'])
    # Group events in a the same thread using 'comment.diff_hunk' and 'comment.commit_id'
    if len(df) == 1:
        grouped_events = [[df.iloc[0].to_dict()]]
    else:
        grouped_events = [group.to_dict(orient='records') for _, group in df.groupby(['comment.diff_hunk', 'comment.commit_id'])]
    return grouped_events


def get_pr_info(sample):
    pr_info = sample["pull_request_info"]
    head_info = sample["head_repo_info"]
    base_info = sample["base_repo_info"]
    events = sample["events"]

    gh_link = f"https://github.com/{pr_info['repo.name']}/pull/{pr_info['pull_request.number']}"
    
    header = f"""📝 **Title**: {pr_info['pull_request.title']}<br>
    📦 **GitHub Repo**: {pr_info['repo.name']}, PR Number: {pr_info['pull_request.number']}, ID: {pr_info['pull_request.id']}.<br>
    Link: [{gh_link}]({gh_link})"""
    pr_info_html = f"""
    <table style="width:100%">
        <tr><th>Attribute</th><th>Detail</th></tr>
        <tr><td>🧾 <strong>PR Type</strong></td><td>{events[0]['type']}</td></tr>
        <tr><td>🟢 <strong>PR State</strong></td><td>{pr_info['pull_request.state']}</td></tr>
        <tr><td>👤 <strong>PR Author</strong></td><td>{pr_info['pull_request.user.login']}</td></tr>
        <tr><td>🏷️ <strong>Head Branch</strong></td><td>ref: {head_info['pull_request.head.ref']}, label: {head_info['pull_request.head.label']}</td></tr>
        <tr><td>🌳 <strong>Base Branch</strong></td><td>{base_info['pull_request.base.ref']}</td></tr>
    </table>
    """
    return header, pr_info_html


def display_events(sample):
    events = sample["events"]
    grouped_events = create_grouped_events(events)
    original_poster = sample["pull_request_info"]['pull_request.user.login']
    for thread in grouped_events:
        # Start a new thread
        thread_html = '<div class="thread">'
        # Get the first event in the thread as a reference
        first_event = thread[0]
        
        # Add shared parts of the events only once
        text = f"""
        <div class="event">
            <table style="width:100%; border: 0;">
                <tr><td><strong>Event Type</strong></td><td>{first_event['type']}</td></tr>
                <tr><td><strong>User</strong></td><td>{first_event['user.login']} (type :<strong>{first_event['user.type']}</strong>)</td></tr>
                <tr><td><strong>Review State</strong></td><td>{first_event['review.state']}</td></tr>
                <tr><td><strong>From Head</strong></td><td>{first_event['pull_request.head.label']}</td></tr>
            </table>
        </div>
        """
        thread_html += text
        
        # Add the bodies of the comments for each event in the thread
        for event in thread:
            if event['comment.body']:
                is_op = original_poster == event['user.login']
                thread_html += format_body(event['comment.body'], event['user.login'], is_op)
        thread_html += '</div>'
        st.markdown(thread_html, unsafe_allow_html=True)
        if first_event['comment.path']:
            path_html = f"<code style='font-family: monospace; font-weight: bold;'>Path:</code> {first_event['comment.path']}"
            st.markdown(path_html, unsafe_allow_html=True)
        if first_event["comment.diff_hunk"]:
            focus_diff, full_diff = show_diff_hunk(first_event["comment.diff_hunk"], first_event["comment.original_position"])
            if not focus_diff:
                components.html(full_diff)
            else:
                components.html(focus_diff)
                with st.expander("View Full diff hunk"):
                    components.html(full_diff)
        st.markdown("---")

def custom_css():
    st.markdown("""
    <style>
        .thread {
            border: 1px solid #ccc;
            padding: 10px;
            margin: 10px;
            background-color: #f9f9f9;
            width: 100%;
        }
        .event {
            border-bottom: 1px solid #eee;
            padding: 10px;
            width: 100%;
        }
        .event-header {
            font-size: 0.9em;
            color: #666;
        }
    </style>
    """, unsafe_allow_html=True)

custom_css()


#st.set_page_config(page_icon=":laptop:", layout="wide")
st.markdown(f"""\
    # GitHub Code Reviews Inspection 🔍
    In this space you can inspect code reviews from GitHUb Pull Requests. Note that some may have empty text (e.g approval of a PR without a code comment).
    You can find the dataset at [bigcode/code_reviews_sample](https://huggingface.co/datasets/bigcode/code_reviews_sample)
    """
    )
example_index = st.number_input(f"Example (0 to {size-1}):", min_value=0, max_value=size-1, value=0, step=1)

header, pr_info_html = get_pr_info(ds[example_index])
st.subheader("PR information")
st.markdown(header, unsafe_allow_html=True)
st.markdown(pr_info_html, unsafe_allow_html=True)
st.markdown("<br>", unsafe_allow_html=True)
st.subheader("Code review events")
event_blocks = display_events(ds[example_index])


In [ ]:
ValueError: The features can't be aligned because the key pull_request_info of features {'pull_request.guid': Value(dtype='string', id=None), 'pull_request.code_review_events': Value(dtype='string', id=None), 'pull_request.events': Value(dtype='string', id=None), 'pull_request.issue_events': Value(dtype='string', id=None), 'bucket': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None), 'pull_request_info': {'org.id': Value(dtype='int64', id=None), 'public': Value(dtype='bool', id=None), 'pull_request.additions': Value(dtype='int64', id=None), 'pull_request.body': Value(dtype='string', id=None), 'pull_request.changed_files': Value(dtype='int64', id=None), 'pull_request.closed_at': Value(dtype='null', id=None), 'pull_request.comments': Value(dtype='int64', id=None), 'pull_request.commits': Value(dtype='int64', id=None), 'pull_request.created_at': Value(dtype='string', id=None), 'pull_request.deletions': Value(dtype='int64', id=None), 'pull_request.guid': Value(dtype='string', id=None), 'pull_request.id': Value(dtype='int64', id=None), 'pull_request.merged_at': Value(dtype='null', id=None), 'pull_request.merged_by.login': Value(dtype='null', id=None), 'pull_request.milestone.description': Value(dtype='null', id=None), 'pull_request.milestone.number': Value(dtype='null', id=None), 'pull_request.milestone.title': Value(dtype='null', id=None), 'pull_request.number': Value(dtype='int64', id=None), 'pull_request.review_comments': Value(dtype='int64', id=None), 'pull_request.state': Value(dtype='string', id=None), 'pull_request.title': Value(dtype='string', id=None), 'pull_request.user.id': Value(dtype='int64', id=None), 'pull_request.user.login': Value(dtype='string', id=None), 'repo.id': Value(dtype='int64', id=None), 'repo.name': Value(dtype='string', id=None)}, 'head_repo_info': {'pull_request.head.label': Value(dtype='string', id=None), 'pull_request.head.ref': Value(dtype='string', id=None), 'pull_request.head.repo.default_branch': Value(dtype='string', id=None), 'pull_request.head.repo.description': Value(dtype='null', id=None), 'pull_request.head.repo.homepage': Value(dtype='null', id=None), 'pull_request.head.repo.language': Value(dtype='string', id=None), 'pull_request.head.repo.license.name': Value(dtype='null', id=None), 'pull_request.head.repo.name': Value(dtype='string', id=None), 'pull_request.head.repo.owner.login': Value(dtype='string', id=None), 'pull_request.head.repo.owner.type': Value(dtype='string', id=None), 'pull_request.head.repo.private': Value(dtype='bool', id=None), 'pull_request.head.repo.stargazers_count': Value(dtype='int64', id=None), 'pull_request.head.sha': Value(dtype='string', id=None), 'pull_request.head.user.login': Value(dtype='string', id=None), 'pull_request.head.user.type': Value(dtype='string', id=None)}, 'base_repo_info': {'pull_request.base.label': Value(dtype='string', id=None), 'pull_request.base.ref': Value(dtype='string', id=None), 'pull_request.base.repo.default_branch': Value(dtype='string', id=None), 'pull_request.base.repo.description': Value(dtype='null', id=None), 'pull_request.base.repo.forks_count': Value(dtype='int64', id=None), 'pull_request.base.repo.homepage': Value(dtype='null', id=None), 'pull_request.base.repo.language': Value(dtype='string', id=None), 'pull_request.base.repo.license.name': Value(dtype='null', id=None), 'pull_request.base.repo.name': Value(dtype='string', id=None), 'pull_request.base.repo.open_issues_count': Value(dtype='int64', id=None), 'pull_request.base.repo.owner.login': Value(dtype='string', id=None), 'pull_request.base.repo.owner.type': Value(dtype='string', id=None), 'pull_request.base.repo.private': Value(dtype='bool', id=None), 'pull_request.base.repo.stargazers_count': Value(dtype='int64', id=None), 'pull_request.base.repo.watchers_count': Value(dtype='int64', id=None), 'pull_request.base.sha': Value(dtype='string', id=None), 'pull_request.base.user.login': Value(dtype='string', id=None), 'pull_request.base.user.type': Value(dtype='string', id=None), 'pull_request.comments': Value(dtype='int64', id=None), 'pull_request.label.name': Value(dtype='null', id=None), 'pull_request.review_comments': Value(dtype='int64', id=None)}, 'events': [{'action': Value(dtype='string', id=None), 'created_at': Value(dtype='timestamp[us, tz=UTC]', id=None), 'issues_comments': {'action': Value(dtype='string', id=None), 'author': Value(dtype='null', id=None), 'comment': Value(dtype='null', id=None), 'comment_id': Value(dtype='null', id=None), 'datetime': Value(dtype='null', id=None), 'type': Value(dtype='string', id=None)}, 'review_comments': {'actor.id': Value(dtype='int64', id=None), 'actor.login': Value(dtype='string', id=None), 'comment.author_association': Value(dtype='null', id=None), 'comment.body': Value(dtype='null', id=None), 'comment.commit_id': Value(dtype='null', id=None), 'comment.created_at': Value(dtype='null', id=None), 'comment.diff_hunk': Value(dtype='null', id=None), 'comment.id': Value(dtype='null', id=None), 'comment.in_reply_to_id': Value(dtype='null', id=None), 'comment.line': Value(dtype='null', id=None), 'comment.original_commit_id': Value(dtype='null', id=None), 'comment.original_line': Value(dtype='null', id=None), 'comment.original_position': Value(dtype='null', id=None), 'comment.original_start_line': Value(dtype='null', id=None), 'comment.path': Value(dtype='null', id=None), 'comment.position': Value(dtype='null', id=None), 'comment.side': Value(dtype='null', id=None), 'comment.start_line': Value(dtype='null', id=None), 'comment.start_side': Value(dtype='null', id=None), 'comment.updated_at': Value(dtype='null', id=None), 'review.author_association': Value(dtype='null', id=None), 'review.body': Value(dtype='null', id=None), 'review.commit_id': Value(dtype='null', id=None), 'review.id': Value(dtype='null', id=None), 'review.state': Value(dtype='null', id=None), 'review.submitted_at': Value(dtype='null', id=None), 'user.login': Value(dtype='null', id=None), 'user.type': Value(dtype='null', id=None)}, 'type': Value(dtype='string', id=None)}]} has unexpected type - {'org.id': Value(dtype='int64', id=None), 'public': Value(dtype='bool', id=None), 'pull_request.additions': Value(dtype='int64', id=None), 'pull_request.body': Value(dtype='string', id=None), 'pull_request.changed_files': Value(dtype='int64', id=None), 'pull_request.closed_at': Value(dtype='null', id=None), 'pull_request.comments': Value(dtype='int64', id=None), 'pull_request.commits': Value(dtype='int64', id=None), 'pull_request.created_at': Value(dtype='string', id=None), 'pull_request.deletions': Value(dtype='int64', id=None), 'pull_request.guid': Value(dtype='string', id=None), 'pull_request.id': Value(dtype='int64', id=None), 'pull_request.merged_at': Value(dtype='null', id=None), 'pull_request.merged_by.login': Value(dtype='null', id=None), 'pull_request.milestone.description': Value(dtype='null', id=None), 'pull_request.milestone.number': Value(dtype='null', id=None), 'pull_request.milestone.title': Value(dtype='null', id=None), 'pull_request.number': Value(dtype='int64', id=None), 'pull_request.review_comments': Value(dtype='int64', id=None), 'pull_request.state': Value(dtype='string', id=None), 'pull_request.title': Value(dtype='string', id=None), 'pull_request.user.id': Value(dtype='int64', id=None), 'pull_request.user.login': Value(dtype='string', id=None), 'repo.id': Value(dtype='int64', id=None), 'repo.name': Value(dtype='string', id=None)} (expected either {'org.id': Value(dtype='null', id=None), 'public': Value(dtype='bool', id=None), 'pull_request.additions': Value(dtype='int64', id=None), 'pull_request.body': Value(dtype='string', id=None), 'pull_request.changed_files': Value(dtype='int64', id=None), 'pull_request.closed_at': Value(dtype='null', id=None), 'pull_request.comments': Value(dtype='int64', id=None), 'pull_request.commits': Value(dtype='int64', id=None), 'pull_request.created_at': Value(dtype='string', id=None), 'pull_request.deletions': Value(dtype='int64', id=None), 'pull_request.guid': Value(dtype='string', id=None), 'pull_request.id': Value(dtype='int64', id=None), 'pull_request.merged_at': Value(dtype='null', id=None), 'pull_request.merged_by.login': Value(dtype='null', id=None), 'pull_request.milestone.description': Value(dtype='null', id=None), 'pull_request.milestone.number': Value(dtype='null', id=None), 'pull_request.milestone.title': Value(dtype='null', id=None), 'pull_request.number': Value(dtype='int64', id=None), 'pull_request.review_comments': Value(dtype='int64', id=None), 'pull_request.state': Value(dtype='string', id=None), 'pull_request.title': Value(dtype='string', id=None), 'pull_request.user.id': Value(dtype='int64', id=None), 'pull_request.user.login': Value(dtype='string', id=None), 'repo.id': Value(dtype='int64', id=None), 'repo.name': Value(dtype='string', id=None)} or Value("null").


In [29]:
import json
res = json.loads(small_ds[0]['pull_request.events'])
res

[{'type': 'PullRequestEvent',
  'action': 'opened',
  'actor.login': 'dependabot[bot]',
  'actor.id': 49699333,
  'user.login': None,
  'user.id': None,
  'user.type': None,
  'repo.name': 'AbdElrahmanMuhammedNasr/WuzuufMasr',
  'repo.id': 210433834,
  'public': True,
  'created_at': '2022-12-10T03:27:08Z',
  'org.id': None,
  'org.login': None,
  'pull_request.id': 1157080683,
  'pull_request.number': 35,
  'pull_request.state': 'open',
  'pull_request.title': 'Bump qs from 6.5.2 to 6.5.3',
  'pull_request.body': 'Bumps [qs](https://github.com/ljharb/qs) from 6.5.2 to 6.5.3.\n<details>\n<summary>Changelog</summary>\n<p><em>Sourced from <a href="https://github.com/ljharb/qs/blob/main/CHANGELOG.md">qs\'s changelog</a>.</em></p>\n<blockquote>\n<h2><strong>6.5.3</strong></h2>\n<ul>\n<li>[Fix] <code>parse</code>: ignore <code>__proto__</code> keys (<a href="https://github-redirect.dependabot.com/ljharb/qs/issues/428">#428</a>)</li>\n<li>[Fix] <code>utils.merge</code>: avoid a crash with a 

In [65]:
issues = issues[0]["events"]
issues

[{'action': 'opened',
  'author': 'hillc-usgs',
  'comment': None,
  'comment_id': None,
  'datetime': '2021-06-24T17:23:03Z',
  'description': 'This PR makes nldi_flowtools able to work with the new pygeoapi restructure, and makes it installable directly into the new tool. The processors are now contained within the library for nldi_flowtools directly, which makes it far simpler to roll out the plugin without needing coding modifications to the USGS pygeoapi tool.',
  'title': 'pygeoapi_plugins refit',
  'type': 'issue'},
 {'action': 'created',
  'author': 'rmcd-mscb',
  'comment': "@Anders-Hopkins - I merged Cliff's changes to keep things moving but you might want to review the changes for yourself when you get back.  ",
  'comment_id': 868826717.0,
  'datetime': '2021-06-25 20:51:35+00:00',
  'description': None,
  'title': None,
  'type': 'comment'}]

In [25]:
for i in range(3, 20):
    row = small_ds[i]
    events = load_json(row["pull_request.events"])
    reviews = load_json(row["pull_request.code_review_events"])
    issues = load_json(row["pull_request.issue_events"])
    if reviews:
        print(i)
        break

10


In [37]:
row = small_ds[10]
events = load_json(row["pull_request.events"])
reviews = load_json(row["pull_request.code_review_events"])
issues = load_json(row["pull_request.issue_events"])
print(f"len events {len(events)}, len reviews {len(reviews)}, len issues {len(issues)}")

len events 2, len reviews 1, len issues 1


In [39]:
events = load_json(row["pull_request.events"])
reviews = load_json(row["pull_request.code_review_events"])
issues = load_json(row["pull_request.issue_events"])

In [40]:
L = events + reviews + issues

In [130]:
events = load_json(row["pull_request.events"])
reviews = load_json(row["pull_request.code_review_events"])
issues = load_json(row["pull_request.issue_events"])
assert len(issues) == 1
issues_events = issues[0]["events"]
# for each events in each category group all events sorted by "created_at" in one list
for e in issues_events:
    e["created_at"] = parse(e["datetime"])
    del e["datetime"]
events = [update_datetime(e) for e in events]
reviews = [update_datetime(e) for e in reviews]
all_events = sorted(
    events + reviews + issues_events,
    key=lambda x: x["created_at"]
)

pr_info = {k: events[0][k] for k in pull_request_info_cols}
head_info = {k: events[0][k] for k in head_info_cols}
base_info = {k:  events[0][k] for k in base_info_cols}
# each comment should have "comments" and "review_comments" fields with "extra_review_info" field
comments = [{"type": e["type"],
            "action": e["action"],
            "created_at": e["created_at"],
            "review_comments":  get_review_info(e),
            "issues_comments": get_issue_info(e)} for e in all_events]
new_row = {"pull_request_info": pr_info, "head_repo_info": head_info, "base_repo_info": base_info, "events": comments}

In [131]:
new_row.keys()

dict_keys(['pull_request_info', 'head_repo_info', 'base_repo_info', 'events'])

In [146]:
pr_info = new_row["pull_request_info"]
res = f"**GitHub Repo**: {pr_info['repo.name']}, PR Number: {pr_info['pull_request.number']}, ID: {pr_info['pull_request.id']}"
gh_link = f"https://github.com/{pr_info['repo.name']}/pull/{pr_info['pull_request.number']}"
res += f"\n**GitHub Link**: {gh_link}"
print(res)
for i in range(len(new_row["events"])):
    e = new_row["events"][i]
    print("-" * 100)
    print(f"Type: {e['type']}, action: {e['action']}, created_at: {e['created_at']}")
    action = e['action']

    if e['type'] in ["issue", "comment"]:
        e = e["issues_comments"]
        print(f"Author {e['author']} did {e['action']}:\n{e['comment']}")

    elif e['type'] in ["PullRequestEvent", "PullRequestReviewCommentEvent", "PullRequestReviewEvent"]:
        reviews = e["review_comments"]
        print(f"Author {reviews['actor.login']} with association {reviews['review.author_association']} did {action}")
        if reviews['review.body']:
            print(f"Review:\n{reviews['review.body']}")
        if reviews['comment.body']:
            print(f"Comment:\n{reviews['comment.body']}")
        if reviews['comment.diff_hunk']:
            print(f"Diff hunk:\n{reviews['diff_hunk']}")
            print(f"File path {reviews['path']}")
    else:
        print("OTHER")
        print(e["type"])


**GitHub Repo**: ACWI-SSWD/nldi_flowtools, PR Number: 4, ID: 677298606
**GitHub Link**: https://github.com/ACWI-SSWD/nldi_flowtools/pull/4
----------------------------------------------------------------------------------------------------
Type: issue, action: opened, created_at: 2021-06-24 17:23:03+00:00
Author hillc-usgs did opened:
None
----------------------------------------------------------------------------------------------------
Type: PullRequestEvent, action: opened, created_at: 2021-06-24 17:23:04+00:00
Author hillc-usgs with association None did opened
----------------------------------------------------------------------------------------------------
Type: PullRequestReviewEvent, action: created, created_at: 2021-06-25 20:50:41+00:00
Author rmcd-mscb with association NONE did created
Review:
Thanks Cliff - Anders has been out this week, to keep things moving I'll merge the request and leave the branch for him to view when he gets back.  
----------------------------------

In [144]:
pr_info

{'pull_request.base.label': 'ACWI-SSWD:master',
 'pull_request.base.ref': 'master',
 'pull_request.base.sha': '4ce49143e7ce6e473554c3ebf7335a23d91ca91c',
 'pull_request.base.user.login': 'ACWI-SSWD',
 'pull_request.base.user.type': 'Organization',
 'pull_request.base.repo.owner.login': 'ACWI-SSWD',
 'pull_request.base.repo.owner.type': 'Organization',
 'pull_request.base.repo.license.name': 'BSD 3-Clause "New" or "Revised" License',
 'pull_request.base.repo.default_branch': 'master',
 'pull_request.base.repo.description': None,
 'pull_request.base.repo.language': 'Python',
 'pull_request.base.repo.watchers_count': 3,
 'pull_request.base.repo.open_issues_count': 1,
 'pull_request.base.repo.forks_count': 0,
 'pull_request.base.repo.name': 'nldi_flowtools',
 'pull_request.base.repo.homepage': None,
 'pull_request.base.repo.stargazers_count': 3,
 'pull_request.base.repo.private': False,
 'pull_request.comments': 0,
 'pull_request.review_comments': 0,
 'pull_request.label.name': None}

In [145]:
events

[{'type': 'PullRequestEvent',
  'action': 'opened',
  'actor.login': 'hillc-usgs',
  'actor.id': 84474574,
  'user.login': None,
  'user.id': None,
  'user.type': None,
  'repo.name': 'ACWI-SSWD/nldi_flowtools',
  'repo.id': 365244721,
  'public': True,
  'created_at': datetime.datetime(2021, 6, 24, 17, 23, 4, tzinfo=tzlocal()),
  'org.id': 17301770,
  'org.login': 'ACWI-SSWD',
  'pull_request.id': 677298606,
  'pull_request.number': 4,
  'pull_request.state': 'open',
  'pull_request.title': 'pygeoapi_plugins refit',
  'pull_request.body': 'This PR makes nldi_flowtools able to work with the new pygeoapi restructure, and makes it installable directly into the new tool. The processors are now contained within the library for nldi_flowtools directly, which makes it far simpler to roll out the plugin without needing coding modifications to the USGS pygeoapi tool.',
  'pull_request.user.login': 'hillc-usgs',
  'pull_request.user.id': 84474574,
  'pull_request.author_association': 'NONE',
  

In [54]:
events[0]["created_at"]
issues[0]["events"][0]["datetime"]

'2021-06-24T17:23:03Z'

In [60]:
issues[0]["events"][1]


{'action': 'created',
 'author': 'rmcd-mscb',
 'comment': "@Anders-Hopkins - I merged Cliff's changes to keep things moving but you might want to review the changes for yourself when you get back.  ",
 'comment_id': 868826717.0,
 'datetime': '2021-06-25 20:51:35+00:00',
 'description': None,
 'title': None,
 'type': 'comment'}

In [56]:
issues[0].keys()

dict_keys(['repo', 'org', 'issue_id', 'issue_number', 'pull_request', 'events'])

In [ ]:
all_events = sorted(
    events + reviews + issues,
    key=lambda x: x["created_at"]
)
pr_info = {k: all_events[-1][k] for k in pull_request_info_cols}
head_info = {k: all_events[-1][k] for k in head_info_cols}
base_info = {k:  all_events[-1][k] for k in base_info_cols}
# each comment should have "comments" and "review_comments" fields with "extra_review_info" field
comments = [{"comments": e["pull_request.comments"],
            "review_comments": e["pull_request.review_comments"],
            "extra_review_info":  get_extra_review_info(e)} for e in all_events]
new_row = {"pr_info": pr_info, "head_info": head_info, "base_info": base_info, "comments": comments}

In [38]:
new_row = merge_events(row)

KeyError: 'created_at'

In [35]:
issues

{'repo': 'ACWI-SSWD/nldi_flowtools',
 'org': 'ACWI-SSWD',
 'issue_id': 929448726,
 'issue_number': 4,
 'pull_request': {'number': 4.0,
  'repo': 'nldi_flowtools',
  'user_login': 'ACWI-SSWD'},
 'events': [{'action': 'opened',
   'author': 'hillc-usgs',
   'comment': None,
   'comment_id': None,
   'datetime': '2021-06-24T17:23:03Z',
   'description': 'This PR makes nldi_flowtools able to work with the new pygeoapi restructure, and makes it installable directly into the new tool. The processors are now contained within the library for nldi_flowtools directly, which makes it far simpler to roll out the plugin without needing coding modifications to the USGS pygeoapi tool.',
   'title': 'pygeoapi_plugins refit',
   'type': 'issue'},
  {'action': 'created',
   'author': 'rmcd-mscb',
   'comment': "@Anders-Hopkins - I merged Cliff's changes to keep things moving but you might want to review the changes for yourself when you get back.  ",
   'comment_id': 868826717.0,
   'datetime': '2021-06

In [ ]:

# for each events in each category group all events sorted by "created_at" in one list
# then merge all three lists
# then sort by "created_at"
all_events = sorted(
    events + reviews + issues,
    key=lambda x: x["created_at"]
)
pr_info = {k: all_events[-1][k] for k in pull_request_info_cols}
head_info = {k: all_events[-1][k] for k in head_info_cols}
base_info = {k:  all_events[-1][k] for k in base_info_cols}
# each comment should have "comments" and "review_comments" fields with "extra_review_info" field
comments = [{"comments": e["pull_request.comments"],
            "review_comments": e["pull_request.review_comments"],
            "extra_review_info":  get_extra_review_info(e)} for e in all_events]

In [ ]:
pull_request_info_cols = [
    "repo.name",
    "repo.id",
    "org.id",
    "public",
    "pull_request.id",
    "pull_request.guid",
    "pull_request.number",
    "pull_request.title",
    "pull_request.body",
    "pull_request.state",
    "pull_request.user.login",
    "pull_request.user.id",
    "pull_request.created_at",
    "pull_request.closed_at",
    "pull_request.merged_at",
    "pull_request.merged_by.login",
    "pull_request.milestone.title",
    "pull_request.milestone.description",
    "pull_request.milestone.number",
    # commits
    'pull_request.commits',
    'pull_request.additions',
    'pull_request.deletions',
    # changed files
    'pull_request.changed_files',
]

comments = [
    'pull_request.comments',
    'pull_request.review_comments',
    # for PR event
    'pull_request.label.name',
    # review events only
    'review.state',
    'review.id', 
    'review.body', 
    'review.commit_id', 
    'review.submitted_at', 
    'review.author_association', '
]

head_info_cols = [
    "pull_request.head.label",
    "pull_request.head.ref",
    "pull_request.head.user.login",
    "pull_request.head.user.type",
    "pull_request.head.repo.owner.login",
    "pull_request.head.repo.owner.type",
    "pull_request.head.repo.license.name",
    "pull_request.head.sha",
    'pull_request.head.repo.name',
    'pull_request.head.repo.owner.login',
    'pull_request.head.repo.homepage',
    'pull_request.head.repo.description',
    'pull_request.head.repo.language',
    'pull_request.head.repo.stargazers_count',
    'pull_request.head.repo.license.name',
    'pull_request.head.repo.default_branch',
    'pull_request.head.repo.private'
]
base_info_cols = [
    "pull_request.base.label",
    "pull_request.base.ref",
    "pull_request.base.sha",
    "pull_request.base.user.login",
    "pull_request.base.user.type",
    "pull_request.base.repo.owner.login",
    "pull_request.base.repo.owner.type",
    "pull_request.base.repo.license.name",
    "pull_request.base.repo.default_branch",
    "pull_request.base.repo.description",
    "pull_request.base.repo.language",
    "pull_request.base.repo.watchers_count",
    "pull_request.base.repo.open_issues_count",
    "pull_request.base.repo.forks_count",
    'pull_request.base.repo.name',
    'pull_request.base.repo.owner.login',
    'pull_request.base.repo.homepage',
    'pull_request.base.repo.description',
    'pull_request.base.repo.language',
    'pull_request.base.repo.stargazers_count',
    'pull_request.base.repo.private',
]
# drop "repo.name", "repo.id", "public" so they are not duplicated and keep relevant columns that might change
event_cols = [
    col
    for col in df.columns
    if (not col.startswith("pull_request."))
    and col not in ["repo.name", "repo.id", "public"]
] + [
    "pull_request.head.label",
    "pull_request.head.ref",
    "pull_request.head.sha",
    "pull_request.title",
]